# Week 8 Homework

***Due (pushed to your GitHub branch) on 11/8 by 11:59 pm***

## Mark duplicates

Use Picard Tools to mark the duplicates of the HISAT and STAR alignments. Store the logs in a new directory.

In [6]:
%mkdir logs_hw
%mkdir dedup_hw

!picard MarkDuplicates -I /data/users/waldhacw6865/BIOL343/2_genome_exploration/beegenome/genome.fa -M logs_hw/star_duplicates -O dedup_hw/star.bam

mkdir: cannot create directory ‘logs_hw’: File exists
mkdir: cannot create directory ‘dedup_hw’: File exists
Error: LinkageError occurred while loading main class picard.cmdline.PicardCommandLine
	java.lang.UnsupportedClassVersionError: picard/cmdline/PicardCommandLine has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0


## Counting

Use featureCounts to count reads. You may need to explicitly set the features and meta-features with `-t` and `-g`.

In [4]:
!featureCounts -T 32 \
    /data/classes/2024/fall/biol343/course_files/dedup/star.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/beegenome/genome_annotations.gtf \
    -o star_counts.tsv \
    --verbose


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           star.bam                                         ||
||                                                                            ||
||             Output file : star_counts.tsv                     

## Report

Use MultiQC to aggregate a report that incorporates all steps of the pipeline.

In [ ]:
!multiqc --force -d ../5_fastq/fastq/qc/ ../5_fastq/trimmed/qc/ ../6_alignment ../7_alignment_qc .

## Snakemake Pipeline (optional)

1. Copy your `Snakefile` and `config.yaml` from `7_alignment_qc` to `8_counting`.  
2. In the `Snakefile`, add the following rules:  
    a. `mark_duplicates` - use Picard Tools to mark the duplicate reads in the STAR and HISAT alignments.  
    b. `count` - use featureCounts to count reads in the alignments with duplicates marked.  
    c. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.